In [1]:
!python ../src/utils.py
%reload_ext autoreload

In [2]:
import numpy as np
from collections import namedtuple
# Import all required libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import ROOT as rt
import pandas as pd
from ROOT import VecOps
import utils

# Enable JSROOT 
%jsroot on

from analysis import df , HMNCSBR, TRUNCSBR

print("cwd:", os.getcwd())

cwd: /home/bothsides/projects/optimizing_DEDx_estimator/notebooks


In [3]:


Metrics = namedtuple("Metrics",
                     ["core_res", "full_res", "tail_frac"])

def resolution_metrics(mpvs,
                       core_low=3.8, core_high=15.0,
                       tail_cut=20.0):
    """
    Parameters
    ----------
    mpvs : 1-D array of per-track MPV values (MeV/cm)
    core_low, core_high : float
        Core window for the dE/dx resolution proxy.
    tail_cut : float
        Threshold that defines the high-tail region.

    Returns
    -------
    Metrics(core_res, full_res, tail_frac)
        core_res  : σ68 / μ in [core_low, core_high)
        full_res  : σ68 / μ over the entire distribution
        tail_frac : fraction with MPV > tail_cut
    """
    mpvs = np.asarray(mpvs)
    if mpvs.size == 0:
        raise ValueError("Empty MPV array")

    # ---- Core band ---------------------------------------------------------
    core = mpvs[(mpvs >= core_low) & (mpvs < core_high)]
    if core.size == 0:
        raise ValueError("No entries inside the core band")

    q16_c, q84_c = np.percentile(core, [16, 84])
    sigma68_core = 0.5 * (q84_c - q16_c)
    mu_core      = core.mean()
    core_res     = sigma68_core / mu_core

    # ---- Full distribution -------------------------------------------------
    q16_f, q84_f = np.percentile(mpvs, [16, 84])
    sigma68_full = 0.5 * (q84_f - q16_f)
    mu_full      = mpvs.mean()
    full_res     = sigma68_full / mu_full

    # ---- Tail fraction -----------------------------------------------------
    tail_frac = np.mean(mpvs > tail_cut)

    return Metrics(core_res, full_res, tail_frac)



In [4]:
print(f"Available estimators:")
print(f"Harmonic means: {HMNCSBR}")
print(f"Truncated means: {TRUNCSBR}")

Available estimators:
Harmonic means: ['DeDx_IhStrip1', 'DeDx_IhStrip', 'DeDx_IhStrip3', 'DeDx_IhStrip4']
Truncated means: ['DeDx_ItStrip0', 'DeDx_ItStrip5', 'DeDx_ItStrip10', 'DeDx_ItStrip15', 'DeDx_ItStrip20', 'DeDx_ItStrip25', 'DeDx_ItStrip30', 'DeDx_ItStrip35', 'DeDx_ItStrip']


In [5]:
cluster = df.AsNumpy(["cluster_DeDxStrip"])["cluster_DeDxStrip"]
results = utils.fit_mpv(cluster)

Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1
Info in <ROOT::Math::ParameterSettings>: lower/upper bounds outside current parameter value. The value will be set to (low+up)/2 
Info in <ROOT::Math::ParameterSettings>: lower/upper bounds outside current parameter value. The value will be set to (low+up)/2 
Info in <ROOT::Math::ParameterSettings>: lower/upper bounds outside current parameter value. The value will be set to (low+up)/2 
Info in <ROOT::Math::ParameterSettings>: lower/upper bounds outside current parameter value. The value will be set to (low+up)/2 
Error in <Minuit2>: VariableMetricBuilder Initial matrix not pos.def.
Error in <Minuit2>: VariableMetricBuilder Initial matrix not pos.def.
Info in <ROOT::Math::ParameterSettings>: lower/upper bounds outside current parameter value. The value will be set to (low+up)/2 
Info in <ROOT::Math::ParameterSettings>: lower/upper bounds outside current parameter value. The value will be set to (low+up)/2 
Error in

In [6]:
neg_tracks = results["neg_tracks"]
neg_track_labels = results["neg_mpvs"]
params = results["params"]


print(len(neg_tracks))

for label, track in zip(neg_track_labels, neg_tracks):
  print(f"{label} : {track}")

18
Event 36888 Trk 2 : { 2.87084f, 7.24237f, 6.13317f, 10.3733f, 18.6288f, 136.122f }
Event 37009 Trk 1 : { 3.93640f, 6.40171f, 3.93247f, 3.06810f, 3.73513f, 2.05625f, 2.60408f, 59.3439f }
Event 37077 Trk 1 : { 7.23092f, 3.21012f, 3.43804f, 99.2167f, 7.71873f, 0.972959f }
Event 37658 Trk 5 : { 3.92032f, 9.37611f, 70.5981f, 4.60633f, 2.58087f, 2.54820f, 3.98561f, 2.05819f }
Event 42271 Trk 0 : { 7.37285f, 3.14832f, 3.37351f, 82.5019f, 2.60977f, 2.95203f }
Event 42577 Trk 0 : { 7.71427f, 7.27343f, 168.580f, 12.7205f, 8.81630f, 6.51776f, 5.73063f, 7.74582f, 6.12025f, 8.00448f, 6.17840f, 6.74174f, 6.46979f, 4.83757f, 4.48795f, 6.70362f }
Event 43057 Trk 4 : { 3.15743f, 3.43331f, 4.28202f, 9.58706f, 2.82304f, 3.58343f, 2.14246f, 3.52100f, 76.7960f }
Event 43106 Trk 0 : { 2.55813f, 2.74532f, 3.77468f, 2.71405f, 4.86658f, 3.15082f, 2.68290f, 3.83715f, 3.05725f, 3.02199f, 4.60036f, 3.08001f, 63.1572f, 6.19792f, 5.65915f, 3.15677f, 2.88726f }
Event 43249 Trk 2 : { 19.8500f, 16.0639f, 7.21180f, 

In [7]:


# ------------------------------------------------------------------
# 1.  Harmonic-2 (Ih estimator)
# ------------------------------------------------------------------
ih_vecs = df.AsNumpy(["DeDx_IhStrip"])["DeDx_IhStrip"]   # list / array of per-event vectors

# flatten to 1-D NumPy array
mpv_h2 = np.concatenate(ih_vecs).astype(float)

# ------------------------------------------------------------------
# 2.  40 % truncated-mean (last Trunc column = DeDx_ItStrip)
# ------------------------------------------------------------------
trunc_vecs = df.AsNumpy(["DeDx_ItStrip"])["DeDx_ItStrip"]

mpv_trunc40 = np.concatenate(trunc_vecs).astype(float)

mpv_landau  = [mpv for mpv, _ in params]

print("mpv_h2 shape     :", mpv_h2.shape)
print("mpv_trunc40 shape:", mpv_trunc40.shape)
print("mpv_landau shape :", np.array(mpv_landau).shape)


mpv_h2 shape     : (11074,)
mpv_trunc40 shape: (11074,)
mpv_landau shape : (9902,)


In [8]:
def describe_array(name, arr):
    """Print type, length, and shape (if ndarray)."""
    print(f"{name}:")
    print(f"  type   → {type(arr)}")
    try:
        print(f"  length → {len(arr)}")
    except TypeError:
        print("  length → n/a (object not iterable)")
    if hasattr(arr, "shape"):
        print(f"  shape  → {arr.shape}")
    print()

# -----------------------------------------------------------------
describe_array("mpv_h2",      mpv_h2)
describe_array("mpv_trunc40", mpv_trunc40)
describe_array("mpv_landau",  mpv_landau)


mpv_h2:
  type   → <class 'numpy.ndarray'>
  length → 11074
  shape  → (11074,)

mpv_trunc40:
  type   → <class 'numpy.ndarray'>
  length → 11074
  shape  → (11074,)

mpv_landau:
  type   → <class 'list'>
  length → 9902



In [9]:


for tag, arr in [("Harmonic-2", mpv_h2),
                 ("Trunc 40 %", mpv_trunc40),
                 ("Landau-MPV", mpv_landau)]:
    m = resolution_metrics(arr)
    print(f"{tag:12s}  core σ/μ={m.core_res:6.3f}  "
          f"full σ/μ={m.full_res:6.3f}  "
          f"tail%={100*m.tail_frac:5.2f}")

Harmonic-2    core σ/μ= 0.312  full σ/μ= 0.485  tail%= 0.15
Trunc 40 %    core σ/μ= 0.296  full σ/μ= 0.492  tail%= 0.14
Landau-MPV    core σ/μ= 0.336  full σ/μ= 0.492  tail%= 0.35
